In [ ]:
import pandas as pd
import numpy as np
import math
import statistics
from dateutil.parser import parse
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from scipy import stats

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 1. Загрузка данных и предобработка

## Выбор турниров

In [ ]:
results_pkl = np.load('gdrive/My Drive/Advanced_ML/results.pkl',allow_pickle=True)

In [ ]:
tournaments_pkl = np.load('gdrive/My Drive/Advanced_ML/tournaments.pkl',allow_pickle=True)
tournaments_np = np.array([list(item.values()) for item in tournaments_pkl.values()])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
del tournaments_pkl

In [ ]:
train_date = parse('2018-12-31T23:59:59+03:00')
test_date_1 = parse('2019-12-31T23:59:59+03:00')
test_date_2 = parse('2020-12-31T23:59:59+03:00')
train_list = []
test_list = []

In [ ]:
len_tour = len(tournaments_np)

In [ ]:
for i in range(len_tour):
  tournaments_np[i][2] = parse(tournaments_np[i][2])

In [ ]:
#find id tournament
unique_id = []
for i in range(len_tour):
  if tournaments_np[i][2] > train_date and tournaments_np[i][2] <= test_date_1:
    train_list.append([tournaments_np[i][0], tournaments_np[i][1], tournaments_np[i][4]['name']])
    if tournaments_np[i][4]['id'] not in unique_id:
      unique_id.append(tournaments_np[i][4]['id'])
  else:
    if tournaments_np[i][2] > test_date_1 and tournaments_np[i][2] < test_date_2:
      test_list.append([tournaments_np[i][0], tournaments_np[i][1], tournaments_np[i][4]['name']])
      if tournaments_np[i][4]['id'] not in unique_id:
        unique_id.append(tournaments_np[i][4]['id'])

In [ ]:
unique_id

[3, 8, 2, 6, 5]

In [ ]:
#del tournaments_np

In [ ]:
len(test_list)

418

In [ ]:
len(train_list)

687

In [ ]:
#[5042, {'id': 2, 'name': 'Обычный'}]
#[4772, {'id': 3, 'name': 'Синхрон'}]
#[5462, {'id': 5, 'name': 'Общий зачёт'}]
#[5394, {'id': 6, 'name': 'Строго синхронный'}]
#[5009, {'id': 8, 'name': 'Асинхрон'}]

## Таблица сложности вопросов

In [ ]:
#ответ команд и вопросы
def complexity_question(tournaments_list, results_pkl):
  results_team_q = []

  for t in range(len(tournaments_list)):
    j = tournaments_list[t][0]
    for i in range(len(results_pkl[j])):#id_team
      try:
        for q in range(len(results_pkl[j][i]['mask'])):#id_question
          results_team_q.append([str(j) + '_' + str(q), int(results_pkl[j][i]['mask'][q])])
      except:
        pass
  results_team_q = pd.DataFrame(results_team_q, columns = ['id_question','answer'])
  results_questions = results_team_q.groupby(['id_question']).mean()
  results_questions.rename(columns={'answer':'complexity_question'}, inplace=True)
  return results_questions

In [ ]:
results_questions_train = complexity_question(train_list, results_pkl)
results_questions_train.head()

,complexity_question
id_question,
4772_0,0.883117
4772_1,0.779221
4772_10,0.831169
4772_11,0.515152
4772_12,0.974026


In [ ]:
results_questions_train = complexity_question(train_list, results_pkl)
results_questions_test = complexity_question(test_list, results_pkl)
results_questions_train.to_csv('gdrive/My Drive/Advanced_ML/results_questions_train.csv') 
results_questions_test.to_csv('gdrive/My Drive/Advanced_ML/results_questions_test.csv') 

## Таблица игроков и ответы, таблица усредненных результатов игроков

In [ ]:
def results_player_q(tournaments_list, results_pkl, results_questions):
  results_player = []

  for t in range(len(tournaments_list)):
    j = tournaments_list[t][0]
    for i in range(len(results_pkl[j])):#id_team
      for k in range(len(results_pkl[j][i]['teamMembers'])):#id_player
        try:
          for q in range(len(results_pkl[j][i]['mask'])):#id_question
            results_player.append([j, results_pkl[j][i]['team']['id'], results_pkl[j][i]['teamMembers'][k]['player']['id'], \
                      results_pkl[j][i]['teamMembers'][k]['rating'], str(j) + '_' + str(q), int(results_pkl[j][i]['mask'][q])])
        except:
          pass
  results_player_pd = pd.DataFrame(results_player, columns = ['id_tournaments', 'id_team', 'id_player','rating', \
                                                                        'id_question', 'answer'])
  #results_player_pd = results_player_pd.join(results_questions.set_index('id_question'), on='id_question')
  results_player_pd = results_player_pd.join(results_questions, on='id_question')
  results_player_pd['answer_weight'] = results_player_pd['answer'] * results_player_pd['complexity_question']
  return results_player_pd    

In [ ]:
results_player_train_pd = results_player_q(train_list, results_pkl, results_questions_train)
results_player_train_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,answer_weight
0,4772,45556,6212,13507,4772_0,1,0.883117,0.883117
1,4772,45556,6212,13507,4772_1,1,0.779221,0.779221
2,4772,45556,6212,13507,4772_2,1,0.445887,0.445887
3,4772,45556,6212,13507,4772_3,1,0.519481,0.519481
4,4772,45556,6212,13507,4772_4,1,0.878788,0.878788


In [ ]:
results_player_train_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd.csv')

In [ ]:
results_player_test_pd = results_player_q(test_list, results_pkl, results_questions_test)
results_player_test_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,answer_weight
0,4957,49804,30152,14433,4957_0,1,0.282609,0.282609
1,4957,49804,30152,14433,4957_1,1,0.119565,0.119565
2,4957,49804,30152,14433,4957_2,1,0.858696,0.858696
3,4957,49804,30152,14433,4957_3,1,0.576087,0.576087
4,4957,49804,30152,14433,4957_4,1,0.260870,0.260870


In [ ]:
results_player_test_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd.csv')

In [ ]:
player_train_mean = results_player_train_pd.groupby(['id_player']).mean()
del player_train_mean['id_team']
del player_train_mean['answer']
del player_train_mean['id_tournaments']
player_train_mean.head()

,rating,complexity_question,answer_weight
id_player,,,
15,4815.900971,0.450747,0.211222
16,3701.461538,0.442879,0.320499
23,791.000000,0.468409,0.306692
31,4606.075869,0.436949,0.255479
35,4543.964328,0.494833,0.317034


In [ ]:
player_train_mean.to_csv('gdrive/My Drive/Advanced_ML/player_train_mean.csv')

In [ ]:
player_test_mean = results_player_test_pd.groupby(['id_player']).mean()
del player_test_mean['id_team']
del player_test_mean['answer']
del player_test_mean['id_tournaments']
player_test_mean.to_csv('gdrive/My Drive/Advanced_ML/player_test_mean.csv')

## Таблица команды (количество ответов и место)

In [ ]:
def results_team(tournaments_list, results_pkl):
  results_team=[]
  for t in range(len(tournaments_list)):
    j = tournaments_list[t][0]
    for i in range(len(results_pkl[j])):#id_team
      try:
        count = 0
        for q in range(len(results_pkl[j][i]['mask'])):#id_question
          count+= int(results_pkl[j][i]['mask'][q])
        results_team.append([j, results_pkl[j][i]['team']['id'], count])
      except:
        pass
  results_team_pd = pd.DataFrame(results_team, columns = ['id_tournaments','id_team','answer'])
  results_team_dense = results_team_pd.groupby(['id_tournaments'])['answer'].rank(method='dense',ascending=False)
  results_team_pd.loc[:,'dense_rank'] = pd.Series(results_team_dense, index=results_team_pd.index)
  return results_team_pd

In [ ]:
results_team_train_pd = results_team(train_list, results_pkl)
results_team_train_pd.head()

,id_tournaments,id_team,answer,dense_rank
0,4772,45556,28,1.0
1,4772,1030,25,2.0
2,4772,4252,25,2.0
3,4772,5444,25,2.0
4,4772,40931,25,2.0


In [ ]:
results_team_test_pd = results_team(test_list, results_pkl)
results_team_test_pd.head()

,id_tournaments,id_team,answer,dense_rank
0,5414,66120,33,1.0
1,5414,312,31,2.0
2,5414,2,31,2.0
3,5414,72338,30,3.0
4,5414,4032,30,3.0


In [ ]:
results_team_train_pd.to_csv('gdrive/My Drive/Advanced_ML/results_team_train_pd.csv')
results_team_test_pd.to_csv('gdrive/My Drive/Advanced_ML/results_team_test_pd.csv') 

## Таблица команды расширенная (+ рейтинг команды...)

In [ ]:
def results_team_mean(tournaments_list, results_pkl, player_mean, results_team_pd):
  id_tour = []
  id_team = []
  rating = []
  com_que =[]
  ans_w =[]
  max_rating = []
  max_com_que =[]
  max_ans_w =[]

  for t in range(len(tournaments_list)):#id_tournaments
    j = tournaments_list[t][0]
    for i in range(len(results_pkl[j])):#id_team
      try:
        count = 0
        for q in range(len(results_pkl[j][i]['mask'])):#id_question
          count+= int(results_pkl[j][i]['mask'][q])
        ratin = 0
        co_q = 0
        an_w = 0
        max_ratin = -10
        max_co_q = -10
        max_an_w = -10
        count_pl = len(results_pkl[j][i]['teamMembers'])
        for k in range(count_pl): #id_player
          id = results_pkl[j][i]['teamMembers'][k]['player']['id']      
          try:
            ind = player_mean.index[player_mean.id_player == id][0]
            ratin += player_mean['rating'][ind]
            co_q += player_mean['complexity_question'][ind]
            an_w += player_mean['answer_weight'][ind]
            max_ratin = max(player_mean['rating'][ind], max_ratin)
            max_co_q = max(player_mean['complexity_question'][ind], max_co_q)
            max_an_w = max(player_mean['answer_weight'][ind], max_an_w)
          except:
            pass
        if count_pl!= 0 and ratin != 0 and co_q != 0 and an_w !=0:
          id_tour.append(j)
          id_team.append(results_pkl[j][i]['team']['id'])
          rating.append(ratin / count_pl)
          com_que.append(co_q / count_pl)
          ans_w.append(an_w / count_pl)
          max_rating.append(max_ratin)
          max_com_que.append(max_co_q)
          max_ans_w.append(max_an_w)
      except:
        pass     
      
  results_tour_team_mean = pd.DataFrame(
      {
        'id_tournaments': id_tour,        
        'id_team': id_team,
        'rating': rating,
        'complexity_question': com_que,
        'answer_weight': ans_w,
        'max_rating': max_rating,
        'max_complexity_question': max_com_que,
        'max_answer_weight': max_ans_w        
      }
  )
  s = results_team_pd.groupby('id_tournaments')['answer'].transform('max')
  results_team_pd['answer_p'] = results_team_pd['answer'].div(s)
  x_y_raiting = pd.merge(results_tour_team_mean, results_team_pd, how='left')
  return x_y_raiting

In [ ]:
results_team_train_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_team_train_pd.csv')
results_team_train_pd.head()
del results_team_train_pd['Unnamed: 0']
results_team_train_pd.head()

,id_tournaments,id_team,answer,dense_rank
0,4772,45556,28,1.0
1,4772,1030,25,2.0
2,4772,4252,25,2.0
3,4772,5444,25,2.0
4,4772,40931,25,2.0


In [ ]:
player_train_mean = pd.read_csv('gdrive/My Drive/Advanced_ML/player_train_mean.csv')
player_train_mean.head()

,id_player,rating,complexity_question,answer_weight
0,15,4815.900971,0.450747,0.211222
1,16,3701.461538,0.442879,0.320499
2,23,791.000000,0.468409,0.306692
3,31,4606.075869,0.436949,0.255479
4,35,4543.964328,0.494833,0.317034


In [ ]:
x_y_train_raiting = results_team_mean(train_list, results_pkl, player_train_mean, results_team_train_pd)

In [ ]:
x_y_train_raiting.head()

,id_tournaments,id_team,rating,complexity_question,answer_weight,max_rating,max_complexity_question,max_answer_weight,answer,dense_rank,answer_p
0,4772,45556,13322.089781,0.420761,0.354680,13832.452188,0.454860,0.389560,28,1.0,1.000000
1,4772,1030,9748.414632,0.430493,0.329444,12805.756774,0.450324,0.382839,25,2.0,0.892857
2,4772,4252,9098.065811,0.423316,0.316211,9934.840977,0.441281,0.327703,25,2.0,0.892857
3,4772,5444,8429.537756,0.417548,0.286602,9002.047327,0.423141,0.300306,25,2.0,0.892857
4,4772,40931,10284.849720,0.448930,0.359209,12714.947230,0.466370,0.374295,25,2.0,0.892857


In [ ]:
x_y_train_raiting.to_csv('gdrive/My Drive/Advanced_ML/x_y_train_raiting.csv')

In [ ]:
results_team_test_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_team_test_pd.csv')
results_team_test_pd.head()
del results_team_test_pd['Unnamed: 0']
results_team_test_pd.head()

,id_tournaments,id_team,answer,dense_rank
0,5414,66120,33,1.0
1,5414,312,31,2.0
2,5414,2,31,2.0
3,5414,72338,30,3.0
4,5414,4032,30,3.0


In [ ]:
player_test_mean = pd.read_csv('gdrive/My Drive/Advanced_ML/player_test_mean.csv')
player_test_mean.head()

,id_player,rating,complexity_question,answer_weight
0,15,3477.600233,0.432065,0.232886
1,16,3841.000000,0.443854,0.204873
2,23,944.000000,0.384127,0.194444
3,31,5882.538462,0.496695,0.273200
4,35,6268.923913,0.406156,0.238819


In [ ]:
x_y_test_raiting = results_team_mean(test_list, results_pkl, player_test_mean, results_team_test_pd)

In [ ]:
x_y_test_raiting.head()

,id_tournaments,id_team,rating,complexity_question,answer_weight,max_rating,max_complexity_question,max_answer_weight,answer,dense_rank,answer_p
0,5414,66120,8520.322436,0.439939,0.347905,10551.091944,0.467234,0.379066,33,1.0,1.000000
1,5414,312,11592.348323,0.451811,0.357554,12823.869906,0.464574,0.381259,31,2.0,0.939394
2,5414,2,12274.793187,0.466353,0.399107,13574.688372,0.477923,0.415112,31,2.0,0.939394
3,5414,72338,9508.134339,0.470640,0.354091,9880.412894,0.480327,0.367173,30,3.0,0.909091
4,5414,4032,10054.105705,0.461091,0.371010,10241.556054,0.547648,0.504562,30,3.0,0.909091


In [ ]:
x_y_test_raiting.to_csv('gdrive/My Drive/Advanced_ML/x_y_test_raiting.csv')

#2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков.

In [ ]:
results_player_train_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd.csv')

In [ ]:
results_player_test_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd.csv')

In [ ]:
del results_player_train_pd['Unnamed: 0']
del results_player_test_pd['Unnamed: 0']

In [ ]:
results_player_test_pd.head()

,id_team,id_player,rating,id_question,answer,complexity_question,answer_weight
0,49804,30152,14433,4957_0,1,0.282609,0.282609
1,49804,30152,14433,4957_1,1,0.119565,0.119565
2,49804,30152,14433,4957_2,1,0.858696,0.858696
3,49804,30152,14433,4957_3,1,0.576087,0.576087
4,49804,30152,14433,4957_4,1,0.260870,0.260870


In [ ]:
def norm(results_player_pd):
  max_id = max(results_player_pd['id_player'])
  max_rating = max(results_player_pd['rating'])
  results_player_pd['id_player'] = results_player_pd['id_player'] / (max_id + 0.0)
  results_player_pd['rating'] = results_player_train_pd['rating'] / (max_rating + 0.0)
  return results_player_pd

In [ ]:
results_player_train_pd = norm(results_player_train_pd)
results_player_test_pd = norm(results_player_test_pd) 

In [ ]:
model = LogisticRegression(random_state=10)
model.fit(results_player_train_pd[['id_player','rating','complexity_question']], results_player_train_pd[['answer']])


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=10)

In [ ]:
print(model.score(results_player_train_pd[['id_player','rating', 'complexity_question']], results_player_train_pd[['answer']]))

0.7325355480869336


In [ ]:
model.score(results_player_test_pd[['id_player','rating','complexity_question']], results_player_test_pd[['answer']])

0.7193785365161969

#3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе.

###Поэтому:
###предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
###в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [ ]:
x_y_test_raiting = pd.read_csv('gdrive/My Drive/Advanced_ML/x_y_test_raiting.csv')
del x_y_test_raiting['Unnamed: 0']

In [ ]:
x_test = x_y_test_raiting[['id_tournaments', 'id_team','rating','complexity_question','answer_weight', 'max_rating',	'max_complexity_question', 'max_answer_weight']]
y_test = x_y_test_raiting[['answer_p']]

In [ ]:
x_train = x_y_train_raiting[['id_tournaments', 'id_team','rating','complexity_question','answer_weight', 'max_rating',	'max_complexity_question', 'max_answer_weight']]
y_train = x_y_train_raiting[['answer_p']]

In [ ]:
model_r = XGBRegressor()
model_r.fit(x_train, y_train)
print(model_r.score(x_train, y_train))

[14:13:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.6246830233773781


In [ ]:
predict_r = model_r.predict(x_test)
predict_r

array([0.76886535, 0.8469981 , 0.8955965 , ..., 0.251472  , 0.24512988,
       0.16927966], dtype=float32)

In [ ]:
x_test.loc[:,'predict_r'] = pd.Series(predict_r, index=x_test.index)
x_test_ = x_test.groupby(['id_tournaments'])['predict_r'].rank(method='dense',ascending=False)
x_test.loc[:,'predict_dense_rank'] = pd.Series(x_test_, index=x_test.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [ ]:
stats.spearmanr(x_y_test_raiting['dense_rank'],  x_test['predict_dense_rank'])

SpearmanrResult(correlation=0.6158148761422617, pvalue=0.0)

In [ ]:
x_y_test_raiting_ = x_y_test_raiting.groupby('id_tournaments')

{5414: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], 5477: [273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370], 5705: [371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 3

In [ ]:
groups = x_y_test_raiting_.groups

In [ ]:
first_key = list(groups)

In [ ]:
#расчет корреляций по тестовому множеству турниров
spearman_arr = []
kendalltau_arr = []
for i in first_key:
  max_k = max(groups[i])
  min_k = min(groups[i])
  spearman_arr.append(stats.spearmanr(x_y_test_raiting['dense_rank'][min_k:max_k+1], x_test['predict_dense_rank'][min_k:max_k+1])[0])
  kendalltau_arr.append(stats.kendalltau(x_y_test_raiting['dense_rank'][min_k:max_k+1], x_test['predict_dense_rank'][min_k:max_k+1])[0])

In [ ]:
#качество метрики качества на тестовом наборе ранговая корреляция Спирмена
statistics.mean(spearman_arr)

0.7459142452355754

In [ ]:
#качество метрики качества на тестовом наборе ранговая корреляция Кендалла
statistics.mean(kendalltau_arr)

0.5896935815252264

#4. Теперь главное: ЧГК — это всё-таки командная игра.


###Поэтому: 
###предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
###разработайте EM-схему для обучения этой модели, реализуйте её в коде;
###обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

In [ ]:
results_player_train_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd_4.csv')
del results_player_train_pd['Unnamed: 0']
results_player_test_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd_4.csv')
del results_player_test_pd['Unnamed: 0']

In [ ]:
results_player_train_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question
0,4772,45556,6212,13507,4772_0,1,0.883117
1,4772,45556,6212,13507,4772_1,1,0.779221
2,4772,45556,6212,13507,4772_2,1,0.445887
3,4772,45556,6212,13507,4772_3,1,0.519481
4,4772,45556,6212,13507,4772_4,1,0.878788


In [ ]:
results_player_test_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question
0,4957,49804,30152,14433,4957_0,1,0.282609
1,4957,49804,30152,14433,4957_1,1,0.119565
2,4957,49804,30152,14433,4957_2,1,0.858696
3,4957,49804,30152,14433,4957_3,1,0.576087
4,4957,49804,30152,14433,4957_4,1,0.260870


In [ ]:
#удаление игроков с нулевым рейтингом
results_player_train_pd = results_player_train_pd.loc[results_player_train_pd["rating"] != 0]
results_player_test_pd = results_player_test_pd.loc[results_player_test_pd["rating"] != 0]

In [ ]:
results_player_train_pd

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question
0,4772,45556,6212,13507,4772_0,1,0.883117
1,4772,45556,6212,13507,4772_1,1,0.779221
2,4772,45556,6212,13507,4772_2,1,0.445887
3,4772,45556,6212,13507,4772_3,1,0.519481
4,4772,45556,6212,13507,4772_4,1,0.878788
...,...,...,...,...,...,...,...
19292605,6255,75113,212749,14,6255_31,0,0.292722
19292606,6255,75113,212749,14,6255_32,0,0.381132
19292607,6255,75113,212749,14,6255_33,0,0.266307
19292608,6255,75113,212749,14,6255_34,0,0.217251


In [ ]:
s = pd.Series([i for i in range(len(results_player_train_pd))])
results_player_train_pd = results_player_train_pd.set_index([s])
del s
results_player_train_pd['z_p_q'] = pd.Series([0 for i in range(len(results_player_train_pd))])
results_player_train_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q
0,4772,45556,6212,13507,4772_0,1,0.883117,0
1,4772,45556,6212,13507,4772_1,1,0.779221,0
2,4772,45556,6212,13507,4772_2,1,0.445887,0
3,4772,45556,6212,13507,4772_3,1,0.519481,0
4,4772,45556,6212,13507,4772_4,1,0.878788,0


In [ ]:
s = pd.Series([i for i in range(len(results_player_test_pd))])
results_player_test_pd = results_player_test_pd.set_index([s])
del s
results_player_test_pd['z_p_q'] = pd.Series([0 for i in range(len(results_player_test_pd))])
results_player_test_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q
0,4957,49804,30152,14433,4957_0,1,0.282609,0
1,4957,49804,30152,14433,4957_1,1,0.119565,0
2,4957,49804,30152,14433,4957_2,1,0.858696,0
3,4957,49804,30152,14433,4957_3,1,0.576087,0
4,4957,49804,30152,14433,4957_4,1,0.260870,0


In [ ]:
#0 шаг ЕМ алгоритма : предскажем вероятность ответа конкретного игрока
model = LogisticRegression(random_state=10)
model.fit(results_player_train_pd[['id_player','rating','complexity_question']], results_player_train_pd[['answer']])
pred_an = model.predict_proba(results_player_train_pd[['id_player','rating','complexity_question']])
results_player_train_pd['pred_ans'] = pd.Series(pred_an[:,1])
results_player_train_pd['pred_ans_1'] = pd.Series(pred_an[:,0])
del pred_an
pred_an = model.predict_proba(results_player_test_pd[['id_player','rating','complexity_question']])
results_player_test_pd['pred_ans'] = pd.Series(pred_an[:,1])
results_player_test_pd['pred_ans_1'] = pd.Series(pred_an[:,0])
del pred_an
results_player_test_pd.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q,pred_ans,pred_ans_1
0,4957,49804,30152,14433,4957_0,1,0.282609,0,0.620292,0.379708
1,4957,49804,30152,14433,4957_1,1,0.119565,0,0.620292,0.379708
2,4957,49804,30152,14433,4957_2,1,0.858696,0,0.620292,0.379708
3,4957,49804,30152,14433,4957_3,1,0.576087,0,0.620292,0.379708
4,4957,49804,30152,14433,4957_4,1,0.260870,0,0.620292,0.379708


In [ ]:
# создание массива вопросов и игроков, отвечающих на конкретный вопрос 
questions_train_pd = results_player_train_pd.groupby('id_question')
questions_train = questions_train_pd.groups
questions_player_train = list(questions_train)

questions_test_pd = results_player_test_pd.groupby('id_question')
questions_test = questions_test_pd.groups
questions_player_test = list(questions_test)

In [ ]:
questions_train['4772_0']

Int64Index([    0,    36,    72,   108,   144,   180,   216,   252,   288,
              324,
            ...
            41616, 41652, 41688, 41724, 41760, 41796, 41832, 41868, 41904,
            41940],
           dtype='int64', length=1166)

In [ ]:
# если ответ = 0, то и z_p_q = 0
# pred_ans_1 = 1 - pred_ans
# 1. посчитать произведение group(id_tournaments, id_team, id_question)[pred_ans_1]
# если ответ = 0, то z_p_q = (pred_ans_1) / (1 - 1.)
def e_step(df, first_key, groups, pred_ans, z_p_q, pred_ans_1):
  for k in first_key:
    if groups[k][0] > 0:
      div = 1
      for i in groups[k]:        
        div *= df[pred_ans_1][i]
      df.loc[groups[k], z_p_q] = df.loc[groups[k], pred_ans] / (1 - div)
  return df

In [ ]:
results_player_train_pd = e_step(results_player_train_pd, questions_player_train, questions_train, 'pred_ans', 'z_p_q', 'pred_ans_1')

In [ ]:
results_player_train_pd

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q,pred_ans,pred_ans_1
0,4772,45556,6212,13507,4772_0,1,0.883117,0.000000,0.631315,0.368685
1,4772,45556,6212,13507,4772_1,1,0.779221,0.631315,0.631315,0.368685
2,4772,45556,6212,13507,4772_2,1,0.445887,0.631315,0.631315,0.368685
3,4772,45556,6212,13507,4772_3,1,0.519481,0.631315,0.631315,0.368685
4,4772,45556,6212,13507,4772_4,1,0.878788,0.631315,0.631315,0.368685
...,...,...,...,...,...,...,...,...,...,...
15330839,6255,75113,212749,14,6255_31,0,0.292722,0.318869,0.318869,0.681131
15330840,6255,75113,212749,14,6255_32,0,0.381132,0.318869,0.318869,0.681131
15330841,6255,75113,212749,14,6255_33,0,0.266307,0.318869,0.318869,0.681131
15330842,6255,75113,212749,14,6255_34,0,0.217251,0.318869,0.318869,0.681131


In [ ]:
results_player_test_pd = e_step(results_player_test_pd, questions_player_test, questions_test, 'pred_ans', 'z_p_q', 'pred_ans_1')

In [ ]:
results_player_test_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q,pred_ans,pred_ans_1
0,4957,49804,30152,14433,4957_0,1,0.282609,0.000000,0.620292,0.379708
1,4957,49804,30152,14433,4957_1,1,0.119565,0.620292,0.620292,0.379708
2,4957,49804,30152,14433,4957_2,1,0.858696,0.620292,0.620292,0.379708
3,4957,49804,30152,14433,4957_3,1,0.576087,0.620292,0.620292,0.379708
4,4957,49804,30152,14433,4957_4,1,0.260870,0.620292,0.620292,0.379708


In [ ]:
results_player_train_pd.loc[(results_player_train_pd.z_p_q < 0.5), 'z_p_q_round'] = int(0)
results_player_train_pd.loc[(results_player_train_pd.z_p_q >= 0.5), 'z_p_q_round'] = int(1)

In [ ]:
results_player_test_pd.loc[(results_player_test_pd.z_p_q < 0.5), 'z_p_q_round'] = int(0)
results_player_test_pd.loc[(results_player_test_pd.z_p_q >= 0.5), 'z_p_q_round'] = int(1)

In [ ]:
results_player_train_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q,pred_ans,pred_ans_1,z_p_q_round
0,4772,45556,6212,13507,4772_0,1,0.883117,0.000000,0.631315,0.368685,0.0
1,4772,45556,6212,13507,4772_1,1,0.779221,0.631315,0.631315,0.368685,1.0
2,4772,45556,6212,13507,4772_2,1,0.445887,0.631315,0.631315,0.368685,1.0
3,4772,45556,6212,13507,4772_3,1,0.519481,0.631315,0.631315,0.368685,1.0
4,4772,45556,6212,13507,4772_4,1,0.878788,0.631315,0.631315,0.368685,1.0


In [ ]:
# z_p_q через 'id_player','rating','complexity_question'
def m_step(df_train, df_test, pred_ans, pred_ans_1, z_p_q):
  model = LogisticRegression(random_state = 100)
  model.fit(df_train[['id_player','rating','complexity_question']], df_train[[z_p_q]])
  pred_an = model.predict_proba(df_train[['id_player','rating','complexity_question']])
  df_train[pred_ans] = pd.Series(pred_an[:,1])
  df_train[pred_ans_1] = pd.Series(pred_an[:,0])

  pred_an = model.predict_proba(df_test[['id_player','rating','complexity_question']])
  df_test[pred_ans] = pd.Series(pred_an[:,1])
  df_test[pred_ans_1] = pd.Series(pred_an[:,0])
  return df_train, df_test

In [ ]:
results_player_train_pd, results_player_test_pd = m_step(results_player_train_pd, results_player_test_pd,\
                                                         'pred_ans_2', 'pred_ans_1_2', 'z_p_q_round')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
results_player_train_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd_em_1.csv') 
results_player_test_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd_em_1.csv') 

In [ ]:
#2 step e
results_player_train_pd = e_step(results_player_train_pd, questions_player_train, questions_train, 'pred_ans_2', 'z_p_q_2', 'pred_ans_1_2')

In [ ]:
results_player_test_pd = e_step(results_player_test_pd, questions_player_test, questions_test, 'pred_ans_2', 'z_p_q_2', 'pred_ans_1_2')

In [ ]:
ind_nan = results_player_test_pd['z_p_q_2'].isna()
for i in range(len(results_player_test_pd)):
  if ind_nan[i]:
    results_player_test_pd['z_p_q_2'][i] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
results_player_train_pd.loc[(results_player_train_pd.z_p_q < 0.5), 'z_p_q_round_2'] = int(0)
results_player_train_pd.loc[(results_player_train_pd.z_p_q >= 0.5), 'z_p_q_round_2'] = int(1)

In [ ]:
results_player_test_pd.loc[(results_player_test_pd.z_p_q < 0.5), 'z_p_q_round_2'] = int(0)
results_player_test_pd.loc[(results_player_test_pd.z_p_q >= 0.5), 'z_p_q_round_2'] = int(1)

In [ ]:
results_player_train_pd, results_player_test_pd = m_step(results_player_train_pd, results_player_test_pd,\
                                                         'pred_ans_3', 'pred_ans_1_3', 'z_p_q_round_2')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
results_player_train_pd.head()

,id_tournaments,id_team,id_player,rating,id_question,answer,complexity_question,z_p_q,pred_ans,pred_ans_1,z_p_q_round,pred_ans_2,pred_ans_1_2,z_p_q_2,z_p_q_round_2,pred_ans_3,pred_ans_1_3
0,4772,45556,6212,13507,4772_0,1,0.883117,0.000000,0.631315,0.368685,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,4772,45556,6212,13507,4772_1,1,0.779221,0.631315,0.631315,0.368685,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,4772,45556,6212,13507,4772_2,1,0.445887,0.631315,0.631315,0.368685,1.0,1.0,0.0,1.0,1.0,1.0,0.0
3,4772,45556,6212,13507,4772_3,1,0.519481,0.631315,0.631315,0.368685,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,4772,45556,6212,13507,4772_4,1,0.878788,0.631315,0.631315,0.368685,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
results_player_train_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd_em_2.csv') 
results_player_test_pd.to_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd_em_2.csv') 

In [ ]:
results_player_train_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd_em_2.csv') 
results_player_test_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd_em_2.csv') 

In [ ]:
results_player_tour_train = results_player_train_pd.groupby(['id_tournaments', 'id_team']).mean()

In [ ]:
sperman = stats.spearmanr(results_player_tour_train['pred_ans'], results_player_tour_train['answer'])[0]
kendall = stats.kendalltau(results_player_tour_train['pred_ans'], results_player_tour_train['answer'])[0]
print(sperman, kendall)

0.4805831048646366 0.33680408942419643


In [ ]:
sperman = stats.spearmanr(results_player_tour_train['pred_ans_2'], results_player_tour_train['answer'])[0]
kendall = stats.kendalltau(results_player_tour_train['pred_ans_2'], results_player_tour_train['answer'])[0]
print(sperman, kendall)

0.4509007311258246 0.3160735273001841


In [ ]:
sperman = stats.spearmanr(results_player_tour_train['pred_ans_3'], results_player_tour_train['answer'])[0]
kendall = stats.kendalltau(results_player_tour_train['pred_ans_3'], results_player_tour_train['answer'])[0]
print(sperman, kendall)

0.4509007311258246 0.3160735273001841


In [ ]:
results_player_tour_test = results_player_test_pd.groupby(['id_tournaments', 'id_team']).mean()

In [ ]:
sperman = stats.spearmanr(results_player_tour_test['pred_ans'], results_player_tour_test['answer'])[0]
kendall = stats.kendalltau(results_player_tour_test['pred_ans'], results_player_tour_test['answer'])[0]
print(sperman, kendall)

0.423184106062748 0.2943442464981085


In [ ]:
sperman = stats.spearmanr(results_player_tour_test['pred_ans_2'], results_player_tour_test['answer'])[0]
kendall = stats.kendalltau(results_player_tour_test['pred_ans_2'], results_player_tour_test['answer'])[0]
print(sperman, kendall)

0.3972584039941906 0.27656382545462666


In [ ]:
sperman = stats.spearmanr(results_player_tour_test['pred_ans_3'], results_player_tour_test['answer'])[0]
kendall = stats.kendalltau(results_player_tour_test['pred_ans_3'], results_player_tour_test['answer'])[0]
print(sperman, kendall)

0.3972584039941906 0.27656382545462666


# 5. А что там с вопросами?

### Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [ ]:
#ответ команд и вопросы + турниры
def complexity_question(tournaments_list, results_pkl):
  results_team_q = []

  for t in range(len(tournaments_list)):
    j = tournaments_list[t][0]
    for i in range(len(results_pkl[j])):#id_team
      try:
        for q in range(len(results_pkl[j][i]['mask'])):#id_question
          results_team_q.append([tournaments_list[t][0], tournaments_list[t][1], tournaments_list[t][2], str(j) + '_' + str(q), \
                                 int(results_pkl[j][i]['mask'][q])])
      except:
        pass
  results_team_q = pd.DataFrame(results_team_q, columns = ['id_tournaments','name_tournaments', 'type_tournaments','id_question','answer'])
  dct = {
    'id_question': lambda col: col.mode(),
    'id_tournaments': lambda col: col.mode(),
    'name_tournaments': lambda col: col.mode(),
    'type_tournaments': lambda col: col.mode(),
    'answer': 'mean',    
  }
  results_questions = results_team_q.groupby(['id_question']).agg(**{k: (k, v) for k, v in dct.items()})
  results_questions.rename(columns={'answer':'complexity_question'}, inplace=True)
  s = pd.Series([i for i in range(len(results_questions))])
  results_questions = results_questions.set_index([s])
  return results_questions

In [ ]:
results_questions_train = complexity_question(train_list, results_pkl)
results_questions_train.head()

,id_question,id_tournaments,name_tournaments,type_tournaments,complexity_question
0,4772_0,4772,Синхрон северных стран. Зимний выпуск,Синхрон,0.883117
1,4772_1,4772,Синхрон северных стран. Зимний выпуск,Синхрон,0.779221
2,4772_10,4772,Синхрон северных стран. Зимний выпуск,Синхрон,0.831169
3,4772_11,4772,Синхрон северных стран. Зимний выпуск,Синхрон,0.515152
4,4772_12,4772,Синхрон северных стран. Зимний выпуск,Синхрон,0.974026


In [ ]:
results_questions_test = complexity_question(test_list, results_pkl)
results_questions_test.head()

,id_question,id_tournaments,name_tournaments,type_tournaments,complexity_question
0,4957_0,4957,Синхрон Биркиркары,Синхрон,0.282609
1,4957_1,4957,Синхрон Биркиркары,Синхрон,0.119565
2,4957_10,4957,Синхрон Биркиркары,Синхрон,0.760870
3,4957_11,4957,Синхрон Биркиркары,Синхрон,0.260870
4,4957_12,4957,Синхрон Биркиркары,Синхрон,0.543478


In [ ]:
results_questions_train.nlargest(n=20, columns=['complexity_question'])

,id_question,id_tournaments,name_tournaments,type_tournaments,complexity_question
178,5000_19,5000,Чёрная Быль,Синхрон,1.0
183,5000_23,5000,Чёрная Быль,Синхрон,1.0
184,5000_24,5000,Чёрная Быль,Синхрон,1.0
190,5000_3,5000,Чёрная Быль,Синхрон,1.0
195,5000_34,5000,Чёрная Быль,Синхрон,1.0
196,5000_35,5000,Чёрная Быль,Синхрон,1.0
198,5000_5,5000,Чёрная Быль,Синхрон,1.0
1192,5111_13,5111,Открытый чемпионат Кипра,Обычный,1.0
1197,5111_18,5111,Открытый чемпионат Кипра,Обычный,1.0
1210,5111_3,5111,Открытый чемпионат Кипра,Обычный,1.0


In [ ]:
results_questions_test.nlargest(n=20, columns=['complexity_question'])

,id_question,id_tournaments,name_tournaments,type_tournaments,complexity_question
537,5746_1,5746,Открытый Чемпионат Кипра,Обычный,1.0
589,5746_57,5746,Открытый Чемпионат Кипра,Обычный,1.0
1180,5889_1,5889,Славянка,Обычный,1.0
1240,5889_64,5889,Славянка,Обычный,1.0
1277,5891_16,5891,Кубок малой родины,Обычный,1.0
1316,5891_51,5891,Кубок малой родины,Обычный,1.0
1638,5970_22,5970,Январское диминуэндо,Синхрон,1.0
1656,5970_7,5970,Январское диминуэндо,Синхрон,1.0
1665,5973_14,5973,Части Вселенной,Обычный,1.0
1680,5973_28,5973,Части Вселенной,Обычный,1.0


#6. Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли.

In [ ]:
results_player_train_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_train_pd_em_2.csv') 
results_player_test_pd = pd.read_csv('gdrive/My Drive/Advanced_ML/results_player_test_pd_em_2.csv') 

In [ ]:
del results_player_train_pd['Unnamed: 0']
del results_player_test_pd['Unnamed: 0']

In [ ]:
dct = {
    'id_player': lambda col: col.mode(),
    'id_question': 'size',
    'z_p_q': 'mean',    
  }
results_questions_train = results_player_train_pd[['id_player', 'id_question', 'z_p_q']].groupby(['id_player']).agg(**{k: (k, v) for k, v in dct.items()})
s = pd.Series([i for i in range(len(results_questions_train))])
results_questions_train = results_questions_train.set_index([s])
results_questions_train.rename(columns={'id_question':'count_question'}, inplace=True)
results_questions_train.rename(columns={'z_p_q':'z_p_q_rating'}, inplace=True)
results_questions_train.head()

,id_player,count_question,z_p_q_rating
0,15,479,0.553433
1,16,234,0.538278
2,23,36,0.508178
3,31,877,0.551440
4,35,769,0.551236


In [ ]:
#вывод игроков топ 20 по параметру z_p_q_rating
results_questions_train.nlargest(n=20, columns=['z_p_q_rating'])

,id_player,count_question,z_p_q_rating
596,4270,2756,0.638135
864,6212,2986,0.632542
972,7008,4190,0.629657
222,1585,1550,0.627977
324,2421,3132,0.626820
441,3207,3196,0.626635
888,6482,3347,0.626384
66,505,1970,0.626001
2558,18332,3587,0.624182
434,3159,1808,0.623342


In [ ]:
#максимальное количество вопросов
results_questions_train['count_question']. max()

8190

In [ ]:
#вывод игроков топ 20 по параметру z_p_q_rating и количеству вопросов > 5000
results_questions_train.loc[(results_questions_train.count_question > 5000)].nlargest(n=20, columns=['z_p_q_rating'])

,id_player,count_question,z_p_q_rating
571,4121,5240,0.619160
1689,12313,6378,0.615210
3803,27499,6622,0.602457
617,4426,5845,0.599313
3688,26640,6097,0.597700
1715,12511,7915,0.593043
5642,42511,5300,0.590999
2866,20640,8190,0.588966
3530,25475,6092,0.582197
4076,29500,5478,0.576567


In [ ]:
dct = {
    'id_player': lambda col: col.mode(),
    'id_question': 'size',
    'z_p_q': 'mean',    
  }
results_questions_test = results_player_test_pd[['id_player', 'id_question', 'z_p_q']].groupby(['id_player']).agg(**{k: (k, v) for k, v in dct.items()})
s = pd.Series([i for i in range(len(results_questions_test))])
results_questions_test = results_questions_test.set_index([s])
results_questions_test.rename(columns={'id_question':'count_question'}, inplace=True)
results_questions_test.rename(columns={'z_p_q':'z_p_q_rating'}, inplace=True)
results_questions_test.head()

,id_player,count_question,z_p_q_rating
0,15,570,0.554018
1,16,43,0.539717
2,23,36,0.509764
3,31,195,0.560649
4,35,92,0.564588


In [ ]:
#вывод игроков топ 20 по параметру z_p_q_rating
results_questions_test.nlargest(n=20, columns=['z_p_q_rating'])

,id_player,count_question,z_p_q_rating
408,4270,298,0.635171
612,6482,966,0.634965
597,6212,528,0.634208
48,505,377,0.634065
232,2421,595,0.632792
667,7008,769,0.632221
160,1585,448,0.631232
512,5328,657,0.631060
395,4121,989,0.630518
278,2954,996,0.629684


In [ ]:
#максимальное количество вопросов
results_questions_test['count_question']. max()

2143

In [ ]:
#вывод игроков топ 20 по параметру z_p_q_rating и количеству вопросов > 1200
results_questions_test.loc[(results_questions_test.count_question > 1200)].nlargest(n=20, columns=['z_p_q_rating'])

,id_player,count_question,z_p_q_rating
1194,12313,1291,0.623545
995,10145,1327,0.622976
913,9342,1486,0.622909
2951,30152,1475,0.621950
363,3767,1343,0.619426
2096,21233,1412,0.613914
2683,27499,1255,0.611622
2610,26640,1425,0.610484
3335,34395,1336,0.603162
1212,12511,1655,0.601881
